In [1]:
import sys
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from copy import deepcopy
from tqdm import tqdm
import random
from random import shuffle

In [2]:
import cv2
import numpy as np
import random
import os
import re
from tqdm import tqdm
from copy import deepcopy
import sys

import cv2
import numpy as np
from tqdm import tqdm
from multiprocessing import Process, Manager


In [3]:
from IPython.display import clear_output
from matplotlib import pyplot as plt
import collections
%matplotlib inline

def live_plot(data, figsize=(16,8), title=''):
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    plt.imshow(convert_to_rgb(data.astype(int)))
    plt.title(title)
    plt.show()

In [4]:
manager = Manager()  # create only 1 mgr
# prob_dict = manager.dict()

## 读取数据

In [5]:
def convert_to_rgb(img):
    img = deepcopy(img)
    b,g,r = cv2.split(img)  
    img = cv2.merge([r,g,b])
    return img

In [6]:
background = cv2.imread('../../base_data/background.jpeg')
hedge = cv2.imread('../../base_data/hedge.jpeg')

## 生成视频代码

In [7]:
def generate_video(frame_dir=None, video_dir=None, video_name='video.avi'):

    image_num = len([i for i in os.listdir(frame_dir) if '.jpg' in i])
    image_name_list = [f'{i}.jpg' for i in range(image_num)]
    frame = cv2.imread(os.path.join(frame_dir, image_name_list[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(f'{video_dir}/{video_name}', 0, 20, (width, height))

    # video = cv2.VideoWriter(f'{self.video_dir}/{video_name}', 0, 20, (width, height))

    for image in image_name_list:
        video.write(cv2.imread(os.path.join(frame_dir, image)))

    cv2.destroyAllWindows()
    video.release()

## 定义参数

In [8]:
speed_dict = {
    "background": 1,
    "hedge_1": 2,
    "hedge_2": 3,
    "black_bar": 4
}

crop_height = 900
crop_width = 1500
slide_width = 800
camera_width = 250

crop_width_extend = crop_width + 1000

size_1 = camera_width * 2
size_2 = slide_width

dimension = 3
black_bar_width = 10

background_prob = 0.7
noise_average_size = 10

vertical_movement = [0]
horizontal_movement = [1, 2]

In [9]:
background_crop = deepcopy(background[-crop_height:, :crop_width, :])
# live_plot(background_crop)

In [10]:
hedge_crop = deepcopy(hedge[-crop_height:, :crop_width, :])
# live_plot(hedge_crop)

In [11]:
def rgb_mean(image):
    return tuple([np.mean([np.mean(image[:, :, i])]) for i in range(2, -1, -1)])

def initialise_image(image):
    reshape_size = image.shape[0] * image.shape[1]
    a = np.zeros((dimension, reshape_size))
    for i in range(3):
        a[i] = np.array(image[:, :, i]).reshape(1, reshape_size)
    return a

def random_select():
    return random.randint(0, 250), random.randint(0, 750)

def random_radius():
    return random.randint(10, 35)

def mahalanobis(x, mean, cov):
    return np.sqrt(np.linalg.inv(cov).dot(np.transpose(x - mean)).dot((x - mean)))

def cal_ratio_prob(x):
    return 1 - (x/x.sum())

def ratio_soft_max(self, x):
    return ratio_prob(soft_max(x))

def soft_max(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [12]:
hedge_mean = np.array(rgb_mean(hedge_crop))
background_mean = np.array(rgb_mean(background_crop))
hedge_cov = np.cov(initialise_image(hedge_crop))
background_cov = np.cov(initialise_image(background_crop))

## 显示图片

In [13]:
# plt.imshow(convert_to_rgb(background))
# plt.show()
# plt.imshow(convert_to_rgb(hedge))
# plt.show()

In [14]:
background.shape, hedge.shape

((941, 1672, 3), (1128, 1504, 3))

## 定义参数

In [15]:
speed_dict = {
    "background": 1,
    "hedge_1": 2,
    "hedge_2": 3,
    "black_bar": 4
}

crop_height = 900
crop_width = 1500
slide_width = 800
camera_width = 250

crop_width_extend = crop_width + 1000

size_1 = camera_width * 2
size_2 = slide_width

dimension = 3
black_bar_width = 10

background_prob = 0.7
noise_average_size = 10

vertical_movement = [0]
horizontal_movement = [1, 2]

In [16]:
background_crop = deepcopy(background[-crop_height:, :crop_width, :])
# live_plot(background_crop)

In [17]:
hedge_crop = deepcopy(hedge[-crop_height:, :crop_width, :])
# live_plot(hedge_crop)

## 检测位移的算法

In [18]:
def calculate_frame_prob(frame):
    frame_prob = np.zeros(frame.shape)
    for i in range(frame.shape[0]):
        for j in range(frame.shape[1]):
            pixel = frame[i][j]
            hedge_distance = mahalanobis(pixel, hedge_mean, hedge_cov)
            top_distance = mahalanobis(pixel, background_mean, background_cov)
            ratio_prob = cal_ratio_prob(np.array([top_distance, hedge_distance]))[0]
            frame_prob[i][j] = round(ratio_prob)
    return frame_prob

In [19]:
n = 16
m = 10
set_shape = (2*n, 2*n, 3)

In [20]:
frame_dir = '../scene_data/100'

scene_dict = {}
for file_name in os.listdir(frame_dir):
    if file_name[0] != '.':
        num, verti_pos, hori_pos, real_back_prob = file_name.replace('.jpg', '').split('_')
        scene_dict[int(num)] = {'file_name': file_name, 'num':num, 'verti_pos':int(verti_pos), 'hori_pos':int(hori_pos)}


In [21]:
result_compare_list = []

for _ in tqdm(range(1000), ncols=70):
    compare_list = []
    scene_num = random.choice(list(scene_dict.keys()))
    for num in [scene_num, scene_num + 1]:
        file_name = scene_dict[num]['file_name']
        verti_pos = scene_dict[num]['verti_pos']
        hori_pos = scene_dict[num]['hori_pos']
        frame = np.array(cv2.imread(f"{frame_dir}/{file_name}"))
        compare_list.append([frame, (verti_pos, hori_pos), num])
    ground_truth = np.array(compare_list[1][1]) - np.array(compare_list[0][1])

    compare_dict = {}
    frame = compare_list[0][0]
    compare_frame = compare_list[1][0]
    for _ in range(32):
        while 1:
            frame_i, frame_j = random.choice(range(frame.shape[0])), random.choice(range(frame.shape[1]))
            pixels = frame[frame_i - n:frame_i + n, frame_j - n:frame_j + n, :]
            pixels_prob = calculate_frame_prob(pixels)
            if pixels.shape == set_shape and np.sum(pixels_prob) >= set_shape[0] * set_shape[1] * set_shape[2] * 0.8:
                break

        for i in range(frame_i-m, frame_i+m):
            for j in range(frame_j-m, frame_j+m):
                compare_pixels = compare_frame[i - n:i + n,j - n:j + n,:]
                if pixels.shape == set_shape and compare_pixels.shape == set_shape:
    #                 pixels_prob = calculate_frame_prob(pixels)
    #                 compare_pixels_prob = calculate_frame_prob(compare_pixels)
    #                 total_prob = pixels_prob * compare_pixels_prob

                    diff_array = np.array([i for i in ((pixels - compare_pixels)).reshape(-1,1) if i != 0])

                    diff = np.sum(np.power(diff_array,2))/len(diff_array)
                    compare_dict[(frame_i - i, frame_j - j)] = compare_dict.get((frame_i - i, frame_j - j), []) + [diff]
    compare_dict = {k:np.mean(v) for k,v in compare_dict.items()}
    compare_dict = {k: v for k, v in sorted(compare_dict.items(), key=lambda item: item[1])}
    predict = np.array(list(compare_dict.keys())[0])
    result_compare_list.append(list(ground_truth) == list(predict))
    print(ground_truth, predict)
    print('{:.2f}%'.format(sum(result_compare_list)*100/len(result_compare_list)))
    

  0%|                             | 1/1000 [01:03<17:33:50, 63.29s/it]

[-1  2] [-1  7]
0.00%


  0%|                             | 2/1000 [02:04<17:24:18, 62.78s/it]

[-1  1] [-1  2]
0.00%


  0%|                             | 3/1000 [03:08<17:27:25, 63.04s/it]

...

  1%|▎                            | 9/1000 [09:24<17:15:19, 62.68s/it]

[-1  2] [-1  8]
22.22%


  1%|▎                           | 10/1000 [10:27<17:16:00, 62.79s/it]

[2 2] [2 8]
20.00%


KeyboardInterrupt: 